<a href="https://colab.research.google.com/github/sharmatec/Hello-World/blob/master/WikiGooglebasedQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Wikipedia and Google

In [0]:
!pip install wikipedia
!pip install Google-Search-API

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=c7de797f8aa217d3a5c46e436b59d9aff4dec67534e868b357bf45b3432b22a9
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 890kB 14.1MB/s 
     |████████████████████████████████| 245kB 52.9MB/s 
     |████████████████████████████████| 256kB 59.5MB/s 
     |████████████████████████████████| 153kB 44.5MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=4250ce8ac31467b3dbf103e297cabf2bcac287bd0092eebcb300fd8432f40cab
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


Import Libraries

In [0]:
import unicodedata
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk, conlltags2tree, tree2conlltags, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from google import google
import wikipedia
import collections

Download NLTK Datasets

In [0]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Cateforize the Question Based on some specific words

In [0]:
def categorize_question(question):

  question = question.lower().split()
  if question[0] == 'where':
    q_category = 'Location'
  elif 'year'  in question:
    q_category = 'Date'
  elif 'country' in question:
    q_category = 'Country'
  elif question[0] == 'who':
    q_category = 'Person'
  elif question[0] == 'what':
    q_category = 'Definition'
  else:
    q_category = 'None'

  return q_category

Skip the first two tokens

In [0]:
def skip_first_two_tokens(tokens):
  new_tokens = tokens
  if(len(tokens) > 2):
    return tokens[2:]
  return tokens

Apply the lemmatizer

In [0]:
def apply_lemmatizer(tokens):
  lemmatizer = WordNetLemmatizer()
  filtered_tokens = []
  for token in tokens:
    filtered_tokens.append(lemmatizer.lemmatize(token))
  return filtered_tokens

Remove Stopwords

In [0]:
def remove_stopwords(tokens):
  all_stopwords = stopwords.words('english')
  filtered_tokens = [word for word in tokens if not word in all_stopwords]
  return filtered_tokens

Filter the tokens using the above functions

In [0]:
def filter_tokens(tokens):
  filtered_tokens = remove_stopwords(tokens)
  filtered_tokens = apply_lemmatizer(filtered_tokens)
  return filtered_tokens

Named Entity Recognition

In [0]:
def named_entity_recognizer(text):
  ne_tree = (ne_chunk(pos_tag(word_tokenize(text))))
  iob_tagged = tree2conlltags(ne_tree)
  return iob_tagged

Filter the Question

In [0]:
def filter_question(question):
  tokens = TweetTokenizer().tokenize(question)
  new_tokens = skip_first_two_tokens(tokens)

  filtered_tokens = filter_tokens(new_tokens)
  filtered_question = ' '.join(filtered_tokens)
  return filtered_question

Wikipedia Search

In [0]:
def wiki_search(question):
  
  filtered_question = filter_question(question)
  answer = (wikipedia.summary(filtered_question, sentences=1))
  link = wikipedia.page(filtered_question)

  # Providing the link for more information
  answer = answer + '\n For more information follow the link: '+link.url
  return answer

Google Search

In [0]:
def google_search(question):
  top_results = []

  results = google.search(question, 1)
  for result in results:
    top_results.append(result.description)

  search_result = ''.join(top_results)
  print(search_result)

  # Named Entity Tagging
  iob_tagged = named_entity_recognizer(search_result)
  # print('iob_tagged: ', iob_tagged)

  question_category = categorize_question(question)
  if question_category == 'None':
      ans = "Sorry... I don't know the answer!"
  else:
      google_answer = []
      if question_category == 'Person':
          for i in range(len(iob_tagged)):
              # If find a PERSON TAG
              if iob_tagged[i][2] == 'B-PERSON' or iob_tagged[i][2] == 'I-PERSON':
                  # Collect Firstname and Lastname
                  google_answer.append(iob_tagged[i][0] + ' ' + iob_tagged[i+1][0])
      elif question_category == 'Country':
          for i in range(len(iob_tagged)):
              # If find a COUNTRY TAG
              if iob_tagged[i][2] == 'B-GPE'or iob_tagged[i][2] == 'I-GPE':
                  # Collect the name
                  google_answer.append(iob_tagged[i][0])
      elif question_category == 'Location':
          for i in range(len(iob_tagged)):
              # If find a LOCATION TAG
              if iob_tagged[i][2] == 'B-LOCATION'or iob_tagged[i][2] == 'I-LOCATION':
                  # Collect the location name
                  google_answer.append(iob_tagged[i][0])
      elif question_category == 'Date':
          for i in range(len(iob_tagged)):
              # If find a DATE TAG
              if iob_tagged[i][2] == 'B-DATE'or iob_tagged[i][2] == 'I-DATE':
                  # Collect the date
                  google_answer.append(iob_tagged[i][0])
      if not google_answer:
          answer = "Sorry... I don't know the answer! "
      else:
          # Count the Collected Candidates
          candidate_counts = collections.Counter(google_answer)

          # Collect and Sort 5 Most Frequent Candidates 
          # in the Google Snippet 
          most_frequent_candidates = candidate_counts.most_common(5)
          print(most_frequent_candidates)

          # candidate[0]: text
          # candidate[1]: count
          candidate_answers =  [candidate[0] for candidate in most_frequent_candidates]

          for i in range(len(candidate_answers)):
              candidate_answers[i] = 'Candidate Answer '+ str(i+1)+': '+ candidate_answers[i]
          
          candidate_answers = '\n'.join(candidate_answers)
          answer = candidate_answers
  return answer

Answer the Question

In [0]:
def answer_question(question):
  q_category = categorize_question(question)
  print("Question Category: ", q_category)

  # If question was asking about 'Definition' or 
  # 'Location' check try to search on Wikipedia
  if q_category == 'Definition' or q_category == 'Location':
    print('Wikipedia Search')
    answer = wiki_search(question)

  # else try to use google snippet
  else:
    answer = google_search(question)
    print('-'*100)
    print('Google Search')

  return answer

Calling 'answer_question' with different input questions

In [0]:
print(answer_question('Who is the president of United States?'))
print('='*200)
print(answer_question('Who is the president of Iran?'))
print('='*200)
print(answer_question('Where is Tehran?'))
print('='*200)
print(answer_question('Where are the most populated cities in world?'))
print('='*200)
print(answer_question('What is the name of Canada\'s capital city?'))
print('='*200)
print(answer_question('Who directed Gone with the wind?'))
print('='*200)

Question Category:  Person
Donald Trump is the 45th and current president of the United States. He assumed office on January 20, 2017.Grover Cleveland served two non-consecutive terms in office (the only president to have done so) and is therefore counted as the 22nd and 24th president of the United States; the 45th and current president is Donald Trump (since January 20, 2017).Which President served as a lieutenant colonel in the Spanish-American war? Who was the first Democrat elected after the Civil War? Who introduced Social ...The President is both the head of state and head of government of the United States of America, and Commander-in-Chief of the armed forces. Under Article II of ...As the head of the government of the United States, the president is arguably the most powerful government official in the world. The president is elected to a ...The Constitution succinctly defines presidential functions, powers, and responsibilities. The president's chief duty is to make sure tha

In [0]:
#SQUAD Dataset performance evaluation
answer1 = answer_question('What country did the Normans invade in 1169?') 
print('Expected Answer - Ireland')
print('Predicted Answer - ', answer1)

Question Category:  Country
Jump to Short-term impact of the invasion - The Cambro-Norman invasion resulted in the ... and Munster, where there was a larger Norman presence.The spirit of conquest was in the air. Every landless man was looking to make his fortune. Every baron desired, like his viking forefathers, a land where he could ...May 1, 2019 - Invasion on May 1st, 1169, constituted one of the most significant events in Irish history. ... the successor to Adrian IV, for putting manners on the “barbarous nation” of Ireland. The most ... What did the Normans ever do for us?Historians are agreed that English and French involvement in Ireland was an ... Historians are divided as to what to call the people who came to Ireland in 1169. ... Mac Murchada then travelled to Bristol, on the west coast of England, where he ...Feb 17, 2011 - Simon Schama unravels the story of who did what to whom, and why. ... Background to invasion; The Anglo-Normans arrive in Ireland, 1169 on.Dec 1, 2008 - 

In [0]:
print(answer_question('Where are Jersey and Guernsey?')) #Channel Islands
print(answer_question('Who developed the first commercial steam powered device?')) #Thomas Savery
print(answer_question('Who was the first European to travel the Amazon River?')) #Francisco de Orellana

Question Category:  Location
Wikipedia Search
Guernsey ( (listen); Guernésiais: Guernési) is an island in the English Channel off the coast of Normandy.
 For more information follow the link: https://en.wikipedia.org/wiki/Guernsey
Question Category:  Person
Mar 19, 2014 - In 1698, Thomas Savery, an engineer and inventor, patented a machine that could effectively draw water from flooded mines using steam pressure. Savery used principles set forth by Denis Papin, a French-born British physicist who invented the pressure cooker.The first commercial steam-powered device was a water pump, developed in 1698 by Thomas Savery.Jump to Development of the commercial steam engine - Gaspar Schott was the first to describe the ... He installed his steam-powered device on ...Steam engine history dates back to the 1st century AD when the “aeolipile” was described by the ... These were either small steam jacks or escapement devices.It continued to be manufactured until the late 18th century. The first 